# BBC Articles Spinner
* This is a small NLP project target on using Markov Models in building article spinner applications.
* Here, we'll be dealing with a BBC News corpus containing business texts.

In [1]:
# Loading our corpus.
import pandas as pd
import numpy as np

df = pd.read_csv('/kaggle/input/bbc-business/06_bbc_text_cls.csv')
df = df[df.labels=='business']['text'] # As mentioned, we'll be just focusing in business articles.
df.head()

0    Ad sales boost Time Warner profit\n\nQuarterly...
1    Dollar gains on Greenspan speech\n\nThe dollar...
2    Yukos unit buyer faces loan claim\n\nThe owner...
3    High fuel prices hit BA's profits\n\nBritish A...
4    Pernod takeover talk lifts Domecq\n\nShares in...
Name: text, dtype: object

<p style='color:red'> Iniciar tratamento do corpus (lower, tokenização considerando pontuações...)</p>